In [ ]:
import numpy as np
import netCDF4 as nc
from scipy.stats import ttest_ind, ttest_ind_from_stats, ttest_rel
from collections import OrderedDict
import matplotlib.colors as plt_col
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import matplotlib.colors as mcolors
import matplotlib.gridspec as gridspec
import matplotlib.patches as mpatches
from matplotlib.path import Path
import warnings
warnings.filterwarnings("ignore")

In [ ]:
def plot_map(data,data_his):
    ax = fig.add_subplot(spec[nrow,ncol+1], projection=proj)
    ax.add_feature(cfeature.COASTLINE,zorder=11)
    ax.add_feature(cfeature.LAND,zorder=10)
    gl = ax.gridlines(crs=ccrs.PlateCarree(central_longitude=0.0), draw_labels=True, linewidth=1, color='gray', alpha=0.5, linestyle='--')
    gl.xlabel_style = {'size': 14}
    gl.ylabel_style = {'size': 14}
   
    gl.xlabels_top = False
    gl.ylabels_right = False
    if ncol > 0: 
        gl.ylabels_left = False
    if nrow == 0:
        gl.xlabels_bottom = False
    
    vrange = mod_vars[var]['vrange']
    cticks = [-vrange,-vrange/2,0,vrange/2,vrange]
    ctit = mod_vars[var]['units']
        
    #P = ax.pcolormesh(lon_c,lat_c,data,cmap='seismic',transform=ccrs.PlateCarree())    
    P = ax.pcolormesh(lon_c,lat_c,data,vmin=-vrange,vmax=vrange, cmap='seismic',transform=ccrs.PlateCarree())
    #C = ax.contour(lon,lat,data_his,levels = mod_vars[var]['levels'], 
    #               vmin=mod_vars[var]['vmin'], vmax=mod_vars[var]['vmax'],
    #               cmap='binary',transform=ccrs.PlateCarree())

    ax.set_ylim(-45,45)
    ax.set_xlim(-70,115)

    # Labels
    ax.text(134,-24, plot_labels[nlab], fontsize=16, ha='center', va='center',transform=ccrs.PlateCarree(),zorder=12)
    plt.setp(ax.spines.values(), color='black',zorder=12)
    if nrow ==0: 
        # Add dust simulation titles
        ax.set_title(dust_sim[dsim]['title'],fontsize=18,pad=40)
    if ncol == 0:
        # Add Y-axis variable label
        ax.text(-.4, .5, mod_vars[var]['title'], fontsize=16,ha='center', va='center',transform=ax.transAxes)
    #else:
        # Add colorbar
        #pos = ax.get_position()
        #cax = fig.add_axes([pos.x0+1.02*pos.width, pos.y0, .06*pos.width, pos.height])
        #cax2 = fig.add_axes([pos.x0+(1.02*pos.width + .25*pos.width), pos.y0+.3*pos.height/2, .02*pos.width, .7*pos.height])
        #cbar = plt.colorbar(P,cax=cax,ticks=cticks)
        #cbar_2 = plt.colorbar(C,cax=cax)
        #cax.set_ylabel(ctit,rotation=270,labelpad=10)
        #cax.set_ylabel(('Historical '+ mod_vars[var]['units']),rotation=270,labelpad=16)
        #cbar_2.outline.set_visible(False)
        #cax.tick_params(size=0)
        
        

In [ ]:
def plot_map5(data):
    
    ax = fig.add_subplot(spec[nrow,ncol+1], projection=proj)
    ax.add_feature(cfeature.COASTLINE,zorder=11)
    ax.add_feature(cfeature.LAND,zorder=10)
    gl = ax.gridlines(crs=ccrs.PlateCarree(central_longitude=0.0), draw_labels=True, linewidth=1, color='gray', alpha=0.5, linestyle='--')
    gl.xlabel_style = {'size': 14}
    gl.ylabel_style = {'size': 14}
   
    gl.xlabels_top = False
    gl.ylabels_right = False
    if ncol > 0: 
        gl.ylabels_left = False
    if nrow == 0:
        gl.xlabels_bottom = False
    
    vrange = mod_vars[var]['vrange']
    cticks = [-vrange,-vrange/2,0,vrange/2,vrange]
    
    P = ax.pcolormesh(lon_c,lat_c,data,vmin=-vrange,vmax=vrange, cmap='seismic',transform=ccrs.PlateCarree())
    
    ax.set_ylim(-45,45)
    ax.set_xlim(-70,115)
    
    # Labels
    ax.text(134,-24, plot_labels[nlab], fontsize=16, ha='center', va='center',transform=ccrs.PlateCarree(),zorder=12)
    plt.setp(ax.spines.values(), color='black',zorder=12)
    
    if (nrow ==0): 
        # Add dust simulation titles
        ax.set_title('Difference (Dynamic-Static)\n in Climate Change Signal',fontsize=18,pad=20)
    if ncol == 0:
        # Add Y-axis variable label
        ax.text(-.4, 0.5, mod_vars[var]['title'], fontsize=16,ha='center', va='center',transform=ax.transAxes)
    elif ncol>0:
        # Add colorbar
        pos = ax.get_position()
        cax = fig.add_axes([pos.x0+1.02*pos.width, pos.y0, .06*pos.width, pos.height])
        cbar = plt.colorbar(P,cax=cax,ticks=cticks)
        ctit = mod_vars[var]['units']
        cax.set_ylabel(ctit,rotation=270,labelpad=10)

In [ ]:
def depth_weight_mean(z_i, var_vals):
    # Get index of last non-masked element at depth (counts provide the +1 for indexing interfaces)
    #print(var_vals.shape,'Meep')
    if len(var_vals.shape)==3:
        last_int_idx = np.ma.count(var_vals,axis=0)
    elif len(var_vals.shape)==4:
        last_int_idx = np.ma.count(var_vals,axis=1)
    last_depth = z_i[last_int_idx]
    
    # limit if deeper values occur than integration depth (e.g., 200m)
    # last_depth is a 2D array that will serve as a denominator for the layer thickness array
   
    int_max=mod_vars[var]['int_max']
    int_min=mod_vars[var]['int_min']
    
    last_depth[last_depth>int_max]=int_max
    last_depth[last_depth<int_min]=int_min
    # Create "layer_width" Object From Interface-Depth Array
    tmp_dep_i = z_i
    # Making all values below the integration later equal to int_depth will make the difference = 0
    # and, subsequently, the weight will be zero. This will not make the weight zero for areas below the bottom, though
    # for locations that are shallower than the int_depth. 
    tmp_dep_i[tmp_dep_i>int_max]=int_max
    tmp_dep_i[tmp_dep_i<int_min]=int_min
    # difference the array to get layer thickness
    lay_thick = np.diff(tmp_dep_i)
    
    # Broadcast to #dimensions of the 2D "max_depth"
    # NOTE: weights this will sum to more than 1 in places where depth is < int_depth. 
    #       HOWEVER, areas below bottom will be multipled against masked values and not counted in final weighted mean
    if len(var_vals.shape)==3:
        weights = lay_thick[:,None,None]/last_depth
        
    elif len(var_vals.shape)==4:
        weights = np.transpose(lay_thick[:,None,None,None]/last_depth, (1,0,2,3)) 
        
    # MULTIPLY the weights by the temp. 
    weighted_temp = weights*var_vals
    
    if len(var_vals.shape)==3:
        weighted_mean = np.ma.sum(weighted_temp,axis=0)
    elif len(var_vals.shape)==4:
        weighted_mean = np.ma.sum(weighted_temp,axis=1)
    
    return weighted_mean

In [ ]:
def calc_polygon_ts(poly_str):
    x,y = np.meshgrid(lon, lat)
    
    coors=np.hstack((x.reshape(-1, 1), y.reshape(-1,1)))
    poly_path=Path(np.array(polygons[poly_str]['points']))
    poly_mask = poly_path.contains_points(coors).reshape(y.shape[0],x.shape[1])
    
    tmp_vals = np.ma.where(poly_mask==1,var_val_all,np.nan)
    
    poly_ts = np.nanmean(tmp_vals,axis=(2,1))
    
    return poly_ts

In [ ]:
def polygon_stats(poly_str,poly_ts_his,poly_ts_fut):
    
    his_avg = np.mean(poly_ts_his)
    fut_avg = np.mean(poly_ts_fut)
    
    t, p = ttest_rel(poly_ts_his, poly_ts_fut, axis=0, nan_policy='omit')
    print(poly_str)
    print('Historical:',his_avg,'std:',np.std(poly_ts_his))
    print('Future:',fut_avg,'std:', np.std(poly_ts_fut))
    print('p = ', p)
    print(1000*(fut_avg - his_avg)/365, 100*(fut_avg-his_avg)/his_avg,'%')

    print('')
    

In [ ]:
mod_vars = {'intpp':{'3D':False, 'factor': 12*(365*24*60*60),
                     'title':'Depth Integrated\n(0-100m)\n Primary Production','levels':np.arange(75,400+1,100),
                     'vmin':50, 'vmax':400,'vrange': 40,'units':'(gC m$^{-2}$ yr$^{-1}$)'},
            'epc100':{'3D':False, 'factor': 12*(365*24*60*60),
                     'title':'Particulate Organic\n Carbon Flux \n at 100m', 'levels':np.arange(10,40+1,10),
                     'vmin':5, 'vmax':40,'vrange': 8, 'units':'(gC m$^{-2}$ yr$^{-1}$)'},}

dust_sim = {'static':{'dep_var': 'z_i','title': 'Static Dust'},
            'dynamic':{'dep_var':'lev_bnds','title': 'Dynamic Dust'}}

mod_tims = {'1975-2014_his':{'nt1':12*(2014-1850+1),'nt2':12*(1975-1850)},
            '2061-2100_ssp585':{'nt1':12*(2100-1850+1),'nt2':12*(2061-1850)}}

plot_labels = ['a', 'b', 'c', 'd', 'e', 'f']

polygons = {'nino3':{'points': [(210,5),(210,-5),(270,-5),(270,5),(210,5)]},
            'wpac':{'points': [(160,5),(160,-5),(180,-5),(180,5),(180,5)]}}

## Plotting Drift Corrected Climate Change Signal and Difference

In [ ]:
import pickle
proj = ccrs.PlateCarree(central_longitude=180.0)
fig = plt.figure(figsize=(20, 6))
widths = [10,40,40,2,40,8]
spec = fig.add_gridspec(ncols=len(dust_sim)+4, nrows=len(mod_vars),wspace=0.02,hspace=0.02,width_ratios=widths)
nlab=0
for var, nrow in zip(mod_vars,range(len(mod_vars))):
    z_i = None
    print(var)
    
    for dsim, ncol in zip(dust_sim,range(len(dust_sim))):

        # OPEN Pickle file and read in p-value and slope. These are the same across historical and future
        # Time periods for a given dust simulation
        pickle_fil = (var + '_' + dsim + '_' + 'piC_0101-0351_stats.pkl')
        with open(pickle_fil,'rb') as f:  
            slope,p_value = pickle.load(f)
        
        for tim in mod_tims:
            #print(tim)
            ncfil = dsim + var + '_' + tim + '.nc'
            print(ncfil)
            fid = nc.Dataset(ncfil)
            var_val_all = fid.variables[var][:].squeeze()
            
            # 3D adjustment
            if mod_vars[var]['3D']:
                if z_i is None:
                    z_i = fid.variables['z_i'][:].squeeze()
                var_val_all = depth_weight_mean(z_i, var_val_all)
                
            ndim = np.ndim(var_val_all)
            
            # CORRECT FOR DRIFT
            drift_array = slope*(np.arange(var_val_all.shape[0]).reshape((np.append(var_val_all.shape[0],np.ones(ndim-1)).astype(int))) + 
                                 mod_tims[tim]['nt2'])*np.ones(var_val_all.shape)
            
            var_val_all = mod_vars[var]['factor']*(var_val_all - drift_array)
            #var_val_all = mod_vars[var]['factor']*(var_val_all) # W/O drift correction
            # print(var_val_all.shape)
            
            if (nrow+ncol) == 0:
                lon = fid.variables['lon'][:] 
                lat = fid.variables['lat'][:]
                
                lon_mesh, lat_mesh = np.meshgrid(lon,lat)
                
                lon_bnds = fid.variables['lon_bnds'][:] 
                lat_bnds = fid.variables['lat_bnds'][:] 
    
                lon_c = np.append(lon_bnds[:,0],lon_bnds[-1,-1])
                lat_c = np.append(lat_bnds[:,0],lat_bnds[-1,-1])

            
            # When historical, save historical values
            if tim == '1975-2014_his': 
                var_his = np.mean(var_val_all,axis=0)
                nino3_ts_his = calc_polygon_ts('nino3')
                nino4_ts_his = calc_polygon_ts('wpac')
                
            # For future conditions, take difference 
            else:
                var_dif =  np.mean(var_val_all,axis=0)-var_his
                nino3_ts_fut = calc_polygon_ts('nino3')
                nino4_ts_fut = calc_polygon_ts('wpac')
                
                polygon_stats('nino3',nino3_ts_his,nino3_ts_fut)
                polygon_stats('wpac',nino4_ts_his,nino4_ts_fut)
                
                
                plot_map(var_dif,var_his)
                nlab+=1
                
                if dsim == 'static':
                    # Future minus pi-dif
                    stat_dif = var_dif
                else:
                    dyn_dif = var_dif
            
    # Checking difference between static and dynamic simulations
    # Change column index
    ncol+=2
    plot_map5(dyn_dif-stat_dif)
    nlab+=1

plt.savefig('Figure3')